Import regularily generated 7-day average flow data from USGS shapefile and export a formatted shapefile with correct labels for the streams with new low flows

In [1]:
# Import arcpy and other modules
print("importing arcpy")
import arcpy 
from arcpy import env 
from arcpy.sa import *
from urllib.request import *
import time
import tarfile
arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True

importing arcpy


In [2]:
today = time.strftime("%Y%m%d")
url = 'http://waterwatch.usgs.gov/downloads/cov_shps/pa07dstx_shp.tar.gz'
path = 'N:\\Projects\\Drought\\' 
outpath = path+'archive\\USGS_waterwatch_'+today+'_pa07dstx_shp'
outshape = path+'\\shapes\\USGS_7day_flow_ca_'+today+'.shp'
outshape_final = path+'\\shapes\\USGS_7day_flow_ca_'+today+'_final.shp'

In [3]:
# Download the latest shapefile from http://waterwatch.usgs.gov/downloads/cov_shps/pa07dstx_shp.tar.gz

print("Downloading USGS shapefile  to "+outpath)
urlretrieve(url, outpath+'.tar.gz')

('N:\\Projects\\Drought\\archive\\USGS_waterwatch_20161205_pa07dstx_shp.tar.gz',
 <http.client.HTTPMessage at 0xd68c940>)

In [4]:
# Extract the shapefile
print("Extracting USGS shapefile")
tar = tarfile.open(outpath+'.tar.gz')
tar.extractall(outpath)
tar.close()

Extracting USGS shapefile


In [5]:
# extract the California records and reproject to California Albers
print("Selecting California records and reprojecting")
arcpy.FeatureClassToFeatureClass_conversion(outpath+'\\pa07dstx.shp',path+'\\temp','pa07dstx_ca_'+today+'.shp',' "ST" = \'ca\' ')
wgs84 = arcpy.SpatialReference("WGS 1984")
albers = arcpy.SpatialReference("NAD 1983 California (Teale) Albers (Meters)")
arcpy.DefineProjection_management(path+'\\temp\\pa07dstx_ca_'+today+'.shp',wgs84)

arcpy.Project_management(path+'\\temp\\pa07dstx_ca_'+today+'.shp', outshape, albers)

Selecting California records and reprojecting


<Result 'N:\\Projects\\Drought\\\\shapes\\USGS_7day_flow_ca_20161205.shp'>

In [6]:
# update the gage name to a label
print ("Updating the gage labels")
arcpy.AddField_management(outshape, "Label", "TEXT", "","",200)
expression = "replace_all(!STANAME!)"
codeblock ="""def replace_all(text):
    dic = {' NR ':' NEAR ',
           ' NR. ':' NEAR ',
           ' AB ':' ABOVE ',
           ' ABV ':' ABOVE ',
           ' BL ':' BELOW ',
           ' BLW ':' BELOW ',
           ' R ':' RIVER ',
           ' RV ':' RIVER ',
           ' RVR ':' RIVER ',
           ' TRIB ':' TRIBUTARY ',
           ' SL ':' SLOUGH ',
           ' C ':' CREEK ',
           ' CK ':' CREEK ',
           ' CR ':' CREEK ',
            ' CR N ':' CREEK NEAR ',
           ' CH ':' CHANNEL ',
           ' CHAN ':' CHANNEL ',
           ' CNL ':' CANAL ',
           ' CN ':' CANAL ',
           ' FL ' : ' FLUME ',
           ' D ' : ' DRAIN ',
           ' DR ':' DRAIN ',
           ' FLD ' : ' FLOOD ',
           ' A ':' AT ',
           ' MF ':' MIDDLE FORK ',
           ' SF ':' SOUTH FORK ',
           ' NF ':' NORTH FORK ',
           ' EF ':' EAST FORK ',
           ' WF ':' WEST FORK ',
           'E FK ':'EAST FORK ',
           'W FK ':'WEST FORK ',
           ' FK ': ' FORK ',
           ' F ': ' FORK ',
           ' W ': ' WEST ',
           ' N ': ' NORTH ',
           ' E ': ' EAST ',
           ' S ': ' SOUTH ',
           ' BR ':' BRIDGE ',
           ' HWY ':' HIGHWAY ',
           ' ST ':' STREET ',
           ' RD ':' ROAD ',
           ' PKWY ':' PARKWAY ',
           ' SPG ':' SPRING ',
           ' SPR ':' SPRING ',
           ' SP ':' SPRING ',
           ' SPGS, ':' SPRINGS, ',
           ' CYN ':' CANYON ',
           ' LK ':' LAKE ',
           ' LKS ':' LAKES ',
           ' ISL ':' ISLAND ',
           ' V ':' VALLEY ',
           ' MTN ':' MOUNTAIN ',
           ' SND ':' SAND ',
           ' PH ':' POWER HOUSE ',
           ' IT ':' INTAKE ',
           ' OTLT ':' OUTLET ',
           ' OL ':' OUTLET ',
           ' CO ':' COMPANY ',
           ' PT ':' POINT ',
           ' BAS ':' BASIN ',
           ' RES ':' RESERVOIR ',
           ' RE ':' RESERVOIR ',
           ' LO ':' LOWER ',
           ' L ':' LOWER ',
           ' UP ':' UPPER ',
           ' DIV ':' DIVERSION ',
           ' DIVER ':' DIVERSION ',
           ' TU ' : ' TUNNEL ',
           ' PROJ ' : ' PROJECT ',
           ' BNDRY ': ' BOUNDRY ',
           ' PERC ': ' PERCOLATION ',
           ' IRR ':' IRRIGATION ',
           ' IRRIG ':' IRRIGATION ',
           ' FW RL ':' FRESHWATER RELEASE ',
           ' REL ':' RELEASE ',
           ' CTRL ':' CONTROLLED ',
           ' FRM ':' FROM ',
           ' ID ' : ' IRRIGATION DISTRICT ',
           'PVID' : 'PALO VERDE IRRIGATION DISTRICT',
           'P.V.I.D.' : 'PALO VERDE IRRIGATION DISTRICT',
           'P.P.' : 'POWER PLANT',
           ' PP ' : ' POWER PLANT ',
           'SDPP' : 'SIPHON DROP POWER PLANT',
           'DIV.' : 'DIVERSION',
           'YMCWW' : 'YUMA MAIN CANAL WASTEWAY',
           ' WW ' : ' WASTEWAY ',
           'DR0P' : 'DROP',
           'EB CA AQUEDUCT' : 'EAST BRANCH CALIFORNIA AQUEDUCT',
           ' CA AQUEDUCT ' : ' CALIFORNIA AQUEDUCT ',
           'CA AQ XING' : 'CALIFORNIA AQUEDUCT CROSSING',
           'EDWARDS AFB' : 'EDWARDS AIRFORCE BASE',
           ' WC ' : ' WATER COMPANY ',
           ' UWC ' : ' UNION WATER COMPANY ',
           ' CWD ' : ' COUNTY WATER DISTRICT ',
           'MC GEE' : 'MCGEE',
           'UP CONWAY D' : 'UPPER CONWAY DITCH',
           'TRCKEE' : 'TRUCKEE',
           ' TRU. ' : ' TRUCKEE' ,
           'FT DS' : 'FEET DOWNSTREAM',
           'UTICA CA' : 'UTICA CANYON',
           ' AZ-CAL' : ' AZ-CA',
           ' D PRES ' : ' DEL PRESIDIO ',
           ' COND ' : ' CONDUIT ',
           'BAKRSFLD' : 'BAKERSFIELD',
           'JCK.MDW' : 'JACKSON MEADOWS',
           'WQCP' : 'WATER QUALITY CONTROL PLANT',           
           }
    # Replace all the abbreviations
    for i, j in dic.items():
        text = text.replace(i, j)

    # Replace certain letters  at the front of the string with replacements
    if text.startswith('W '):
        text = 'WEST '+text[2:]
    elif text.startswith('E '):
        text = 'EAST '+text[2:]
    elif text.startswith('N '):
        text = 'NORTH '+text[2:]
    elif text.startswith('S '):
        text = 'SOUTH '+text[2:]
    elif text.startswith('L '):
        text = 'LOWER '+text[2:]
    elif text.startswith('M '):
        text = 'MIDDLE '+text[2:]
    elif text.startswith('EF '):
        text = 'EAST FORK '+text[3:]
    elif text.startswith('WF '):
        text = 'WEST FORK '+text[3:]
    elif text.startswith('MF '):
        text = 'MIDDLE FORK '+text[3:]
    elif text.startswith('SF '):
        text = 'SOUTH FORK '+text[3:]
    elif text.startswith('NF '):
        text = 'NORTH FORK '+text[3:]
    elif text.startswith('SB '):
        text = 'SOUTH BRANCH '+text[3:]
    elif text.startswith('NB '):
        text = 'NORTH BRANCH '+text[3:]
    elif text.startswith('EB '):
        text = 'EAST BRANCH '+text[3:]
    elif text.startswith('WB '):
        text = 'WEST BRANCH '+text[3:]
    elif text.startswith('LK '):
        text = 'LAKE '+text[3:]
    elif text.startswith('UP '):
        text = 'UPPER '+text[3:]
    elif text.startswith('CA '):
        text = 'CALIFORNIA '+text[3:]
    elif text.startswith('JW '):
        text = 'J.W. '+text[3:]
    elif text.startswith('LO '):
        text = 'LOWER '+text[3:]
    elif text.startswith('DIV '):
        text = 'DIVERSION '+text[4:]
    elif text.startswith('SCE '):
        text = 'SOUTHERN CALIFORNIA EDDISON '+text[4:]
    elif text.startswith('PGE '):
        text = 'PACIFIC GAS AND ELECTRIC '+text[4:]
    else:
        text = text

    #Convert to title case
    text = text.title()
    
    # Change the capitalization of some of the words
    dic2 = {' Near ':' near ',
            ' Above ':' above ',
            ' Below ':' below ',
            ' Opposite ':' opposite ',
            ' At ':' at ',
            ' Of ':' of ',
            ' Ca-Az':' CA-AZ',
            ' Az-Ca':' AZ-CA',
            ' Az-Cal':' AZ-CA',
            '-Ariz.-Calif':', AZ-CA',
            ' Ariz. Calif.':' AZ-CA',
            ' Cwc ':' CWC ',
            ' Mwc ':' MWC ',
            }
    for i, j in dic2.items():
        text = text.replace(i, j)

    # Capitalize state name at the end of the string
    if text.endswith(' Ca'):
        text = text[:-3]+', CA'
    elif text.endswith(' Az'):
        text = text[:-3]+', AZ'
    elif text.endswith(' Nv'):
        text = text[:-3]+', NV'
    else:
        text = text

    # Replace any double commas with a single comma
    text = text.replace(',,',',')

    # Add parenthesis around the second clause starting with the first near above below or at
    paren_list = [text.find(' above '),text.find(' near '),text.find(' at '),text.find(' below '),text.find(' opposite ')]
    if max(paren_list) == -1:
        text = text
    else:
        neg1_count = paren_list.count(-1)
        for x in range(0,neg1_count):
            paren_list.remove(-1)
        paren = min(paren_list)
        text = text[:(paren + 1)]+'('+text[(paren + 1):]+')'
    
    return text""" 

arcpy.CalculateField_management(outshape, "Label", expression, "PYTHON_9.3", codeblock)

Updating the gage labels


<Result 'N:\\Projects\\Drought\\\\shapes\\USGS_7day_flow_ca_20161205.shp'>

In [7]:
# Create a new field and add the class names to the field
arcpy.AddField_management(outshape, "Class_Name", "TEXT", "","",50)

expression = "reclass(!CLASS!)"
codeblock = """
def reclass(cls):
    if (cls == 1):
        return "New Low"
    elif (cls == 2):
        return "Much Below Normal"
    elif (cls == 3):
        return "Much Below Normal"
    elif (cls == 4):
        return "Below Normal"
    elif (cls == 5):
        return "Normal"
    elif (cls == 6):
        return "Above Normal"
    elif (cls == 7):
        return "Much Above Normal"
    elif (cls == 8):
        return "New High"
    else:
        return "Not Ranked"
"""
arcpy.CalculateField_management(outshape, "Class_Name", expression, "PYTHON_9.3", codeblock)

<Result 'N:\\Projects\\Drought\\\\shapes\\USGS_7day_flow_ca_20161205.shp'>

In [8]:
# Create a new field for the River Name only
arcpy.AddField_management(outshape, "River_Name", "TEXT", "","",50)

expression = "label_split(!Label!)"
codeblock = """
def label_split(label):
    text = label.split(" (")
    return text[0]
"""
arcpy.CalculateField_management(outshape, "River_Name", expression, "PYTHON_9.3", codeblock)

<Result 'N:\\Projects\\Drought\\\\shapes\\USGS_7day_flow_ca_20161205.shp'>

In [9]:
# Create a new field for the Gage location only
arcpy.AddField_management(outshape, "Gage_Loc", "TEXT", "","",150)

expression = "label_split(!Label!)"
codeblock = """
def label_split(label):
    text = label.split(" (")
    if (len(text)>1):
        return "("+text[1]
    else:
        return label
"""
arcpy.CalculateField_management(outshape, "Gage_Loc", expression, "PYTHON_9.3", codeblock)

<Result 'N:\\Projects\\Drought\\\\shapes\\USGS_7day_flow_ca_20161205.shp'>

In [10]:
# Add strategy and ecological value to the gage locations based on huc12 watershed values
join_features = path + "\\gdbs\\Rivers_at_Risk.gdb\\water_rights_huc12"

arcpy.SpatialJoin_analysis(outshape, join_features, outshape_final)

<Result 'N:\\Projects\\Drought\\\\shapes\\USGS_7day_flow_ca_20161205_final.shp'>

In [11]:
# Export to tables
print ("Exporting to geodatabase")
arcpy.TableToTable_conversion(outshape_final, path+"\\gdbs\\Rivers_at_Risk.gdb", 'USGS_7day_flow_ca')

print ("Exporting to CSV")
arcpy.TableToTable_conversion(outshape_final, path+"\\tables", 'USGS_7day_flow_ca.csv')

Exporting to geodatabase
Exporting to CSV


<Result 'N:\\Projects\\Drought\\\\tables\\USGS_7day_flow_ca.csv'>

In [12]:
# Download statewide summaries of flow classes from http://waterwatch.usgs.gov/ww_cont.php?m=pa07d&w=table&r=ca&a=1
# Another optionhttp://waterwatch.usgs.gov/tables/pa07a/pa07a_ca.xls
print ("Downloading historical stream flow comparision to "+path+"\\tables\\USGS_historic_flow.csv")
page = urlopen('http://waterwatch.usgs.gov/ww_cont.php?m=pa07d&w=table&r=ca&a=1')
page_content = page.readlines()

class_list = ["New Low","Much Below Normal","Below Normal","Normal","Above Normal","Much Above Normal","New High"]
f = open(path+'\\tables\\USGS_historic_flow.csv', 'w')
f.write("Date,Class,Class Name,Percent in Class,Total Stations\n")
for i in range(47,len(page_content)-5):
    line = page_content[i].split()
    for j in range(1,8):
        f.write(line[0].decode("utf-8")+","+str(j)+","+class_list[(j-1)]+","+str(int(line[j].decode("utf-8"))*.01)+","+line[8].decode("utf-8")+"\n")
f.close()

In [13]:
# Write a new table with only the percent below normal for the most recent value
f = open(path+'\\tables\\Recent_below_normal.csv', 'w') ;\
f.write("Date,Percent Below Normal\n");\
line = page_content[(len(page_content)-6)].split();\
f.write(line[0].decode("utf-8")+","+str((int(line[1].decode("utf-8"))+int(line[2].decode("utf-8"))+int(line[3].decode("utf-8")))*.01)+"\n");\
f.close()

In [14]:
print(r"Delete N:\Projects\Drought\gdbs\Rivers_at_Risk.mdb\USGS_7day_flow_ca")
print(r"Copy N:\Project\Drought\gdbs\Rivers_at_Risk.gdb\USGS_7day_flow_ca into C:\Arcscratch\Drought\gdbs\Rivers_at_Risk.mdb")

Delete N:\Projects\Drought\gdbs\Rivers_at_Risk.mdb\USGS_7day_flow_ca
Copy N:\Project\Drought\gdbs\Rivers_at_Risk.gdb\USGS_7day_flow_ca into C:\Arcscratch\Drought\gdbs\Rivers_at_Risk.mdb
